In [1]:

import pandas as pd 
import numpy as np

from plotnine import *
from itertools import product
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures



c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def black_scholes_price(S, K, r, T, sigma):
    """Calculate Black Scholes option price."""
  
    d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    price = S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    
    return price




# Generating Data Sample based on Black Scholes

In [3]:
random_state = 42
np.random.seed(random_state)

S = np.arange(40, 61)
K = np.arange(20, 91)
r = np.arange(0, 0.051, 0.01)
T = np.arange(3/12, 2.01, 1/12)
sigma = np.arange(0.1, 0.81, 0.1)

option_prices = pd.DataFrame(
  product(S, K, r, T, sigma),
  columns=["S", "K", "r", "T", "sigma"]
)

option_prices["black_scholes"] = black_scholes_price(
  option_prices["S"].values, 
  option_prices["K"].values, 
  option_prices["r"].values, 
  option_prices["T"].values, 
  option_prices["sigma"].values
)

option_prices = (option_prices
  .assign(
    observed_price=lambda x: (
      x["black_scholes"] + np.random.normal(scale=0.15)
    )
  )
)

In [4]:
train_data, test_data = train_test_split(
  option_prices, 
  test_size=0.01, random_state=random_state
)

preprocessor = ColumnTransformer(
  transformers=[(
    "normalize_predictors", 
     StandardScaler(),
     ["S", "K", "r", "T", "sigma"]
  )],
  remainder="drop"
)

# NN training

In [5]:
max_iter = 1000

deepnnet_model = MLPRegressor(
  hidden_layer_sizes=(10, 5),
  activation="logistic", 
  solver="lbfgs",
  max_iter=max_iter, 
  random_state=random_state
)
                              
deepnnet_pipeline = Pipeline([
  ("preprocessor", preprocessor),
  ("regressor", deepnnet_model)
])

deepnnet_fit = deepnnet_pipeline.fit(
  train_data.drop(columns=["observed_price"]),
  train_data.get("observed_price")
)

KeyboardInterrupt: 